In [1]:
import pandas as pd
import pyreadr
import geopy.distance
from tqdm.notebook import tqdm
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import contextily as ctx
import os


In [5]:
filename_DD = '../data/nextbike/trips_Dresden 2025-01-29_17-48-45.pkl' # 
df_DD = pd.read_pickle(filename_DD)

In [ ]:
# 2024-09-21 // 14.00  DD Saturday

In [ ]:
df_DD.head()

In [ ]:
# pd.to_datetime('2024-09-14 14:30:00')

In [21]:
trips_match_day = df_DD[(df_DD.datetime_return >= pd.to_datetime('2024-09-21 13:30:00')) & (df_DD.datetime_return <= pd.to_datetime('2024-09-21 14:00:00'))]

In [ ]:
trips_match_day

In [22]:
trips_week_before = df_DD[(df_DD.datetime_return >= pd.to_datetime('2024-09-28 13:30:00')) & (df_DD.datetime_return <= pd.to_datetime('2024-09-28 14:00:00'))]

In [ ]:
# trips_week_before

In [ ]:
# Sample list of coordinates (latitude, longitude) in a city (e.g., New York)

# Convert to GeoPandas DataFrame
geometry = [Point(xy) for xy in zip(trips_match_day.lng_return, trips_match_day.lat_return)]
gdf = gpd.GeoDataFrame(trips_match_day, geometry=geometry, crs="EPSG:4326")  

# Reproject to a metric CRS for plotting with basemap
gdf = gdf.to_crs(epsg=3857)


# Plot the points on a map
fig, ax = plt.subplots(figsize=(10, 10))

gdf.plot(ax=ax, marker="o", color='cornflowerblue', markersize=15) # , label='Free-floating bikes'

# Add a basemap (e.g., OpenStreetMap)
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)

# Add labels for each point
# for x, y, coord in zip(gdf.geometry.x, gdf.geometry.y, coordinates):
#     plt.text(x, y, f'{coord}', fontsize=8, ha='left')
ax.axis('off')
# Customize and display the map
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend(loc="upper left")
# plt.savefig(f'/Users/v.sinichenko/Downloads/plots/{city_chosen}_bikes_parked_on_rushhour_{date_chosen}-{time_chosen}.png', format='png', bbox_inches='tight')
plt.show()


In [ ]:
# df_events_other_cities.drop_duplicates()

In [ ]:
# Sample list of coordinates (latitude, longitude) in a city (e.g., New York)

# Convert to GeoPandas DataFrame
geometry = [Point(xy) for xy in zip(trips_week_before.lng_return, trips_week_before.lat_return)]
gdf = gpd.GeoDataFrame(trips_week_before, geometry=geometry, crs="EPSG:4326")  

# Reproject to a metric CRS for plotting with basemap
gdf = gdf.to_crs(epsg=3857)


# Plot the points on a map
fig, ax = plt.subplots(figsize=(10, 10))

gdf.plot(ax=ax, marker="o", color='cornflowerblue', markersize=15) # , label='Free-floating bikes'

# Add a basemap (e.g., OpenStreetMap)
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)

# Add labels for each point
# for x, y, coord in zip(gdf.geometry.x, gdf.geometry.y, coordinates):
#     plt.text(x, y, f'{coord}', fontsize=8, ha='left')
ax.axis('off')
# Customize and display the map
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend(loc="upper left")
# plt.savefig(f'/Users/v.sinichenko/Downloads/plots/{city_chosen}_bikes_parked_on_rushhour_{date_chosen}-{time_chosen}.png', format='png', bbox_inches='tight')
plt.show()
